In [1]:
# determine the optimal pre-match forecast, given elo_diff and s_elo_diff
# step 3: calculate glicko and s_glicko for each player and do the same

In [2]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college (current)/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
from helper_functions import validate_results
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss,accuracy_score

# can test this on our subset of 10,000 matches as well as all matches in the database:
#df = pd.read_csv('../my_data/elo_pbp_with_surface_8_27.csv')
df = pd.read_csv('../my_data/elo_atp_matches_21st_century_9_11.csv')
del df['Unnamed: 0']

# currently looking at 2014 tour-level matches, excluding Davis Cup
df = df[df['match_year']==2014].reset_index(drop=True)
#df = df[df['match_year'].isin([2013,2014,2015,2016])]
df = df[df['tny_name']!='Davis Cup'].reset_index(drop=True)
df.head(2)

,tny_id,tny_name,surface,tny_date,match_year,match_month,p0_name,p1_name,p0_elo,p1_elo,...,winner,match_id,p0_s_kls,p1_s_kls,p0_s_kls_JS,p1_s_kls_JS,p0_sf_s_kls,p1_sf_s_kls,p0_sf_s_kls_JS,p1_sf_s_kls_JS
0,2014-301,Auckland,Hard,2014-01-06,2014,1,Donald Young,Federico Delbonis,1546.540528,1711.117324,...,1,42810,0.573700,0.589324,0.607268,0.623783,0.628710,0.547534,0.632450,0.583904
1,2014-301,Auckland,Hard,2014-01-06,2014,1,Albert Montanes,Santiago Giraldo,1711.293427,1645.679649,...,0,42811,0.660674,0.645451,0.661284,0.647280,0.607995,0.741102,0.625536,0.713346


In [3]:
p = .2
w_s0 = (1-p)*df['p0_s_pct_JS'] + p*df['p0_sf_s_pct_JS']
w_s0[:5]

0    0.572739
1    0.605783
2    0.608118
3    0.631640
4    0.625908
dtype: float64

In [15]:
# try out different weightings of normalized surface serve stats...
probs = []
for p in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
    wgt_s0 = (1-p)*df['p0_s_pct_JS'] + p*df['p0_sf_s_pct_JS']
    wgt_s1 = (1-p)*df['p1_s_pct_JS'] + p*df['p1_sf_s_pct_JS']
    wgt_r0 = (1-p)*df['p0_r_pct_JS'] + p*df['p0_sf_r_pct_JS']
    wgt_r1 = (1-p)*df['p1_r_pct_JS'] + p*df['p1_sf_r_pct_JS']
    df['p0_s_wgt'+str(p)] = df['tny_stats']+(wgt_s0-df['avg_52_s']) - (wgt_r1-df['avg_52_r'])
    df['p1_s_wgt'+str(p)] = df['tny_stats']+(wgt_s1-df['avg_52_s']) - (wgt_r0-df['avg_52_r'])
    
    df['match_prob_wgt_'+str(p)] = [matchProb(row['p0_s_wgt'+str(p)],1-row['p1_s_wgt'+str(p)]) \
                                    for i,row in df.iterrows()]
    probs.append('match_prob_wgt_'+str(p))
    

In [5]:
# generate match probabilities and z-scores for Klaassen method, with and w/o JS estimators
df['match_prob_kls'] = [matchProb(row['p0_s_kls'],1-row['p1_s_kls']) for i,row in df.iterrows()]
df['match_prob_kls_JS'] = [matchProb(row['p0_s_kls_JS'],1-row['p1_s_kls_JS']) for i,row in df.iterrows()]
df['match_prob_sf_kls'] = [matchProb(row['p0_sf_s_kls'],1-row['p1_sf_s_kls']) for i,row in df.iterrows()]
df['match_prob_sf_kls_JS'] = [matchProb(row['p0_sf_s_kls_JS'],1-row['p1_sf_s_kls_JS']) for i,row in df.iterrows()]
#df['match_prob_s_kls'] = [matchProb(row['p0_s_s_kls'],1-row['p1_s_s_kls']) for i,row in df.iterrows()]

from scipy.stats import norm
for col in ['match_prob_kls','match_prob_kls_JS']:
    df[col.replace('prob','z')] = norm.ppf(df[col])
print df.shape

(2409, 103)


In [6]:
print 'elo baseline: ',  sum((df['elo_diff']>0) == df['winner'])/float(len(df))
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff']])
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff']])
print 'surface elo baseline: ', sum((df['sf_elo_diff']>0) == df['winner'])/float(len(df))
print 'elo 538 baseline: ',  sum((df['elo_diff_538']>0) == df['winner'])/float(len(df))
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff_538']])
print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff_538']])
print 'surface elo 538 baseline: ', sum((df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))

elo baseline:  0.69198837692
0.587237989556
0.590338806649
surface elo baseline:  0.686176836862
elo 538 baseline:  0.693233706932
0.594502008319
0.594839978027
surface elo 538 baseline:  0.696554586966


In [7]:
current = (0,0)
for weight in np.arange(11)*.1:
    accuracy = sum(((1-weight)*df['elo_diff_538']+weight*df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))
    if accuracy > current[1]:
        current=weight,accuracy
print 'best s_elo weight: ',current[0],'accuracy =',current[1]

best s_elo weight:  0.9 accuracy = 0.699460356995


In [8]:
# seems worth trying methods to alter match_prob_kls with elo

In [9]:
# match probabilities with surface 

In [17]:
df.columns

Index([u'tny_id', u'tny_name', u'surface', u'tny_date', u'match_year',
       u'match_month', u'p0_name', u'p1_name', u'p0_elo', u'p1_elo',
       ...
       u'match_prob_wgt_0.8', u'p0_s_wgt0.9', u'p1_s_wgt0.9',
       u'match_prob_wgt_0.9', u'match_prob_kls', u'match_prob_kls_JS',
       u'match_prob_sf_kls', u'match_prob_sf_kls_JS', u'match_z_kls',
       u'match_z_kls_JS'],
      dtype='object', length=103)

In [19]:
cols = [['elo_diff_538'],['elo_diff','sf_elo_diff'],['elo_diff_538','sf_elo_diff_538'],\
        ['elo_diff','sf_elo_diff','match_z_kls'],\
        ['elo_diff_538','sf_elo_diff_538','match_z_kls']]
probs = ['match_prob_wgt_0.3',u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS']
n_splits = 5
validate_results(df,probs=probs,lm_columns=cols,n_splits=n_splits)

match_prob_wgt_0.3
accuracy:  0.64589849984
loss:  0.611805413262
match_prob_kls
accuracy:  0.64797491395
loss:  0.649020424059
match_prob_kls_JS
accuracy:  0.654613918099
loss:  0.615572274698
match_prob_sf_kls
accuracy:  0.634269890701
loss:  0.707428503295
match_prob_sf_kls_JS
accuracy:  0.635098040907
loss:  0.638952522705
lm columns:  ['elo_diff_538']
accuracy:  0.689898292803
loss:  0.583513002548
lm columns:  ['elo_diff', 'sf_elo_diff']
accuracy:  0.690734206917
loss:  0.580185808187
lm columns:  ['elo_diff_538', 'sf_elo_diff_538']
accuracy:  0.694048533053
loss:  0.578919361882
lm columns:  ['elo_diff', 'sf_elo_diff', 'match_z_kls']
accuracy:  0.690315818532
loss:  0.580291245266
lm columns:  ['elo_diff_538', 'sf_elo_diff_538', 'match_z_kls']
accuracy:  0.694045082427
loss:  0.578191150638


In [20]:
df.to_csv('../my_data/elo_atp_matches_21st_century_9_12.csv')

In [13]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score
# cols is a list of column sets for logistic regression; 
# probs are model-specific probabilities
def validate_results(df,probs,lm_columns,n_splits=5):
    kfold = KFold(n_splits=n_splits,shuffle=True)
    scores = np.zeros([len(lm_columns)+len(probs),2,n_splits]);i=0
    for train_ind,test_ind in kfold.split(df):
        lm = linear_model.LogisticRegression(fit_intercept = True)
        train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
        for j,prob_col in enumerate(probs):
            y_preds = test_df[prob_col]>.5
            scores[j][0][i]=accuracy_score(test_df['winner'],y_preds)
            scores[j][1][i]=log_loss(test_df['winner'],test_df[prob_col],labels=[0,1])
        
        for k,cols in enumerate(lm_columns):
            lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
            y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
            y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
            scores[len(probs)+k][0][i]=accuracy_score(test_df['winner'],y_preds)
            scores[len(probs)+k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        i+=1

    for j,prob_col in enumerate(probs):
        print prob_col
        print 'accuracy: ', np.mean(scores[j][0])
        print 'loss: ', np.mean(scores[j][1])
    
    for i,cols in enumerate(lm_columns):
        print 'lm columns: ',cols
        print 'accuracy: ', np.mean(scores[len(probs)+i][0])
        print 'loss: ', np.mean(scores[len(probs)+i][1])

In [9]:
# logistic regression on z scores of our probabilities AND elo_diff/s_elo_diff
from scipy.stats import norm
df_all = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
del df_all['Unnamed: 0']
for col in ['sackmann_prob','klaassen_prob','klaassen_prob_a_400']:
    df_all[col.replace('prob','z')] = norm.ppf(df_all[col])
df_all.loc[df_all['sackmann_z']>10,'sackmann_z'] = 10
df_all.loc[df_all['klaassen_z']>10,'klaassen_z'] = 10
df_all.loc[df_all['klaassen_z_a_400']>10,'klaassen_z_a_400'] = 10

In [11]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

# also will want to try ensemble methods at some point...
kfold = KFold(n_splits=5,shuffle=True)
scores = [],[]
cols = ['elo_diff','s_elo_diff','klaassen_z_a_400']
for train_ind,test_ind in kfold.split(df_all):
    lm = linear_model.LogisticRegression(fit_intercept = True)
    train_df,test_df = df_all.loc[train_ind],df_all.loc[test_ind]
    lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
    y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
    y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
    accuracy = accuracy_score(test_df['winner'],y_preds)
    loss = log_loss(test_df['winner'],y_probs,labels=[0,1])
    scores[0].append(accuracy);scores[1].append(loss)
#print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
print 'coefficients: ', lm.coef_[0]
print 'accuracy: ', np.mean(scores[0])
print 'loss: ', np.mean(scores[1])

coefficients:  [ -5.39480879e-04   1.72440109e-03   1.71413858e+00]
accuracy:  0.778048803942
loss:  0.456072287662


In [ ]:
# perform same analysis across entire match dataset, over the decades

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# n_splits = 5
# kfold = KFold(n_splits=n_splits,shuffle=True)
# scores = np.zeros([3,2,n_splits]);i=0
# cols = ['elo_diff','s_elo_diff']
# for train_ind,test_ind in kfold.split(df):
#     lm = linear_model.LogisticRegression(fit_intercept = True)
#     train_df,test_df = df.loc[train_ind],df.loc[test_ind]
#     lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#     y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     y_preds2 = test_df['match_prob_kls']>.5
#     y_preds3 = test_df['match_prob_kls_JS']>.5
#     y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     scores[0][0][i]=accuracy_score(test_df['winner'],y_preds)
#     scores[0][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#     scores[1][0][i]=accuracy_score(test_df['winner'],y_preds2)
#     scores[1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#     scores[2][0][i]=accuracy_score(test_df['winner'],y_preds3)
#     scores[2][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])

    
#     i+=1
# print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
# print 'accuracy: ', np.mean(scores[0][0])
# print 'loss: ', np.mean(scores[0][1])

# print 'kls probabilities'
# print 'accuracy: ', np.mean(scores[1][0])
# print 'loss: ', np.mean(scores[1][1])

# print 'kls JS probabilities'
# print 'accuracy: ', np.mean(scores[2][0])
# print 'loss: ', np.mean(scores[2][1])

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# # cols is a list of all column sets to test; compare with kls pre-match forecasts
# def validate_results(df,columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(columns)+2,2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for k,cols in enumerate(columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        
#         y_preds2 = test_df['match_prob_kls']>.5
#         y_preds3 = test_df['match_prob_kls_JS']>.5
#         scores[len(columns)][0][i]=accuracy_score(test_df['winner'],y_preds2)
#         scores[len(columns)][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#         scores[len(columns)+1][0][i]=accuracy_score(test_df['winner'],y_preds3)
#         scores[len(columns)+1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])
#         i+=1
    
#     for i,cols in enumerate(columns):
#         print 'columns: ',cols
#         #print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
#         print 'accuracy: ', np.mean(scores[i][0])
#         print 'loss: ', np.mean(scores[i][1])
    
#     print 'kls probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)][0])
#     print 'loss: ', np.mean(scores[len(columns)][1])

#     print 'kls JS probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)+1][0])
#     print 'loss: ', np.mean(scores[len(columns)+1][1])